### Note: Possibility of improvements over the choices of p(t), a0, c0

This notebook aims to showcase Ising solvers running simulated bifurcation.

References:
1. https://advances.sciencemag.org/content/5/4/eaav2372
2. https://advances.sciencemag.org/content/7/6/eabe7953

In [1]:
import numpy as np
import scipy as sp
import multiprocessing as mp
import time
import os
from scipy.sparse import csr_matrix

### Setting float precision

In [2]:
flt = np.float32

### SB algorithms

In [3]:
#@nb.njit(parallel=False)
def one_aSB_run(Q_matrix, steps, dt, Kerr_coef, a0, c0, init_y):
    """
    One (adiabatic) simulated bifurcation run over the full pump schedule.
    
    Parameters:
        Q_matrix (2-D array of float): The matrix representing the coupling field of the problem.
                                       Notice that the diagonal elements should be all zero.
        steps (int): The number of iterations.
        dt (float): Time step for the discretized time.
        Kerr_coef (float): The Kerr coefficient.
        a0, c0 (float): Positive constants.
        init_y (1-D array of float): Initial y.
    
    Return: final_state (1-D array of float)
    """
    
    #if np.diagonal(Q_matrix).any():
    #    raise ValueError("Diagonal elements of Q should be all zero.")
    
    #np.random.seed(sd)
    
    N = Q_matrix.shape[0]
    x = flt(np.zeros(N))
    y = flt(init_y)
    
    for k in range(steps):
        x += a0 * y * dt
        y -= (Kerr_coef * x**3 + a0 * (1. - k/steps) * x - c0 * Q_matrix.dot(x)) * dt
        #x_history[k] = x # for analysis purposes
    
    return np.sign(x)

In [4]:
#@nb.njit(parallel=False)
def one_bSB_run(Q_matrix, steps, dt, a0, c0, init_y):
    """
    One ballistic simulated bifurcation run over the full pump schedule.
    
    Parameters:
        Q_matrix (2-D array of float): The matrix representing the local and coupling field of the problem.
                                       Notice that the diagonal elements should be all zero.
        steps (int): The number of iterations.
        dt (float): Time step for the discretized time.
        a0, c0 (float): Positive constants.
        init_y (1-D array of float): Initial y.
    
    Return: final_state (1-D array of float)
    """
    
    #if np.diagonal(Q_matrix).any():
    #    raise ValueError("Diagonal elements of Q should be all zero.")
    
    #np.random.seed(sd)
    
    N = Q_matrix.shape[0]
    x = flt(np.zeros(N))
    y = flt(init_y)
    
    for k in range(steps):
        x += a0 * y * dt
        y -= (a0 * (1. - k/steps) * x - c0 * Q_matrix.dot(x)) * dt # pump increases from 0 to a0 linearly
        for i in range(N): # parallelizable
            if np.abs(x[i]) > 1:
                x[i] = np.sign(x[i])
                y[i] = 0
        #x_history[k] = x # for analysis purposes
    
    return np.sign(x)

In [5]:
#@nb.njit(parallel=False)
def one_dSB_run(Q_matrix, steps, dt, a0, c0, init_y):
    """
    One discrete simulated bifurcation run over the full pump schedule.
    
    Parameters:
        Q_matrix (2-D array of float): The matrix representing the local and coupling field of the problem.
                                       Notice that the diagonal elements should be all zero.
        steps (int): The number of iterations.
        dt (float): Time step for the discretized time.
        a0, c0 (float): Positive constants.
        init_y (1-D array of float): Initial y.
    
    Return: final_state (1-D array of float)
    """
    
    #if np.diagonal(Q_matrix).any():
    #    raise ValueError("Diagonal elements of Q should be all zero.")
    
    #np.random.seed(sd)
    
    N = Q_matrix.shape[0]
    x = flt(np.zeros(N))
    y = flt(init_y)
    
    for k in range(steps):
        x += a0 * y * dt
        y -= (a0 * (1. - k/steps) * x - c0 * Q_matrix.dot(np.sign(x))) * dt
        for i in range(N): # parallelizable
            if np.abs(x[i]) > 1:
                x[i] = np.sign(x[i])
                y[i] = 0
        #x_history[k] = x # for analysis purposes
    
    return np.sign(x)

### Import problem instances
Make sure diagonal elements are all zero. SB algorithms do not work with local fields.

In [23]:
def Ising_from_MaxCut_file(abs_file_path):
    """
    Import an MaxCut problem from file, and turn it into Ising form.
    
    Parameters:
        abs_file_path (str): The absolute file path for the MaxCut instance file.
    
    Return: A CSR sparse matrix in Ising form.
    """
    
    with open(abs_file_path, 'r') as f:
        coef_lst = f.read().split()
    
    return csr_matrix(([int(x) for x in coef_lst[4::3]], ([int(x)-1 for x in coef_lst[2::3]], \
                        [int(x)-1 for x in coef_lst[3::3]])), shape=(int(coef_lst[0]), int(coef_lst[0])))

In [27]:
N = 60
ins = 0
abs_file_path = os.getcwd() + f"/mac_all/rudy/g05_{N}.{ins}" # absolute dir
J = Ising_from_file(abs_file_path)

steps = 200000
total_time = 200
dt = flt(total_time/steps)
Kerr_coef = flt(.1)
a0 = flt(1)
c0 = flt(.3)

np.random.seed(0)
init_y = np.random.uniform(flt(-0.1), flt(0.1), J.shape[0])

In [32]:
# Without numba
start_time = time.time()
fin_state = one_aSB_run(J, steps, dt, Kerr_coef, a0, c0, init_y)
fin_energy = fin_state.dot(J.dot(fin_state))
cut_val = 0.5 * (np.sum(J) - fin_energy)
total_time = time.time() - start_time
print(f'energy: {fin_energy}; cut value: {cut_val}; time: {total_time} s')

energy: -3.0; cut value: 444.0; time: 8.48065996170044 s


In [35]:
def main(N, ins, sd):
    abs_file_path = os.getcwd() + f"/mac_all/rudy/g05_{N}.{ins}" # absolute dir
    J = Ising_from_file(abs_file_path)
    
    np.random.seed(sd)
    init_y = np.random.uniform(flt(-0.1), flt(0.1), J.shape[0])
    
    fin_state = one_aSB_run(J, steps, dt, Kerr_coef, a0, c0, init_y)
    fin_energy = fin_state.dot(J.dot(fin_state))
    return int(0.5 * (np.sum(J) - fin_energy))

In [36]:
for N in [60, 80, 100]:
    for ins in range(10):
        for sd in range(10):
            with open(os.getcwd()+'/out.txt', 'a') as f:
                f.write(f"{N} {ins} {sd} {main(N, ins, sd)}\n")